In [1]:
import pandas as pd
import numpy as np
import torch
import io
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
import time
from sklearn.metrics import f1_score


In [2]:
%ls
prefix = '/data1/tjss/'
# prefix = '/content/drive/My Drive/Summer research /'
doc_level_df = pd.read_csv(prefix+'data/msgs_tec.csv')
word_level_df = pd.read_csv(prefix+'data/NRC_ht_emotion_sentiment.csv')

MeanEmbeddingsFasttext_NRCJoyVsNotJoy_wordLevelBackground(All_Reviews)Shap.ipynb
MeanEmbeddingsFasttext_NRCJoyVsNotJoy_wordLevelBackground(Joy_Reviews_ALL)Shap.ipynb
MeanEmbeddingsFasttext_NRCJoyVsNotJoy_wordLevelBackground(Joy_Reviews)Shap.ipynb
MeanEmbeddingsFasttext_NRCJoyVsNotJoy_wordLevelBackground(Non_Joy_Reviews_ALL)Shap.ipynb
MeanEmbeddingsFasttext_NRCJoyVsNotJoy_wordLevelBackground(Non_Joy_Reviews)Shap.ipynb
SummedEmbeddingsFasttext_NRCJoyVsNotJoy_wordLevelBackground(All_Reviews)Shap.ipynb
SummedEmbeddingsFasttext_NRCJoyVsNotJoy_wordLevelBackground(Joy_Reviews)Shap.ipynb
SummedEmbeddingsFasttext_NRCJoyVsNotJoy_wordLevelBackground(Non_Joy_Reviews)Shap.ipynb


/home/tjss/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#EDA
print(doc_level_df.shape)
print(doc_level_df.emotion.unique())
doc_level_df = doc_level_df.dropna()
print(doc_level_df.shape)
doc_level_df['message'] = doc_level_df['message'].str.lower() 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
lb_make = LabelEncoder()
doc_level_df['emotion_code'] = lb_make.fit_transform(doc_level_df['emotion'])
print(doc_level_df.iloc[:10,:])
new_doc_level_df = doc_level_df[['message', 'emotion_code']]
print(new_doc_level_df.iloc[:10,:])
new_doc_level_df.head(10)

new_doc_level_df.loc[(new_doc_level_df.emotion_code!=3),'emotion_code']=0
new_doc_level_df.loc[(new_doc_level_df.emotion_code==3),'emotion_code']=1

new_doc_level_train, new_doc_level_test = train_test_split(new_doc_level_df, test_size=0.2, random_state=1)
new_doc_level_train, new_doc_level_val = train_test_split(new_doc_level_train, test_size=0.25, random_state=1)

(21051, 3)
['surprise' 'sadness' 'joy' 'disgust' 'fear' 'anger' nan]
(21049, 3)
           message_id                                            message  \
0  145353048817012736  thinks that @melbahughes had a great 50th birt...   
1  144279638024257536  como una expresiã³n tan simple, una sola oraci...   
2  140499585285111809  the moment when you get another follower and y...   
3  145207578270507009  be the greatest dancer of your life! practice ...   
4  139502146390470656  eww.. my moms starting to make her annual rum ...   
5  146042696899887106  if ur heart hurts all the time for tht person ...   
6  145492569609084928  i feel awful, and it's way too freaking early....   
7  145903955229151232  so chuffed for safc fans! bet me dar comes in ...   
8  142717613234069504  making art and viewing art are different at th...   
9  144183822873927680  soooo dooowwwn!! move on, get some sleep... me...   

    emotion  emotion_code  
0  surprise             5  
1   sadness             4  

/home/tjss/miniconda3/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [4]:
print(new_doc_level_train.emotion_code.value_counts())
print(new_doc_level_val.emotion_code.value_counts())
print(new_doc_level_test.emotion_code.value_counts())

0    7706
1    4923
Name: emotion_code, dtype: int64
0    2570
1    1640
Name: emotion_code, dtype: int64
0    2534
1    1676
Name: emotion_code, dtype: int64


In [5]:
print(new_doc_level_train.shape)
print(new_doc_level_val.shape)
print(new_doc_level_test.shape)

(12629, 2)
(4210, 2)
(4210, 2)


In [6]:
# 0-> not Joy
# 1-> Joy


# 0 Anger
# 1 Disgust
# 2 Fear
# 3 Joy
# 4 Sadness
# 5 Surprise

In [7]:
new_doc_level_train.to_csv(prefix+"data/new_doc_level_train.csv")
new_doc_level_val.to_csv(prefix+"data/new_doc_level_val.csv")
new_doc_level_test.to_csv(prefix+"data/new_doc_level_test.csv")

In [8]:
def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = map(float, tokens[1:])
    return data


In [9]:
embeddings = load_vectors(prefix+"data/crawl-300d-2M.vec")

In [10]:
print(type(embeddings))
print(len(embeddings))
for word in embeddings:
  embeddings[word] = [float(i) for i in embeddings[word]]
embeddings_keylist = list(embeddings)
embeddings_vectorlist = list(embeddings.values())
word_to_index={}
index_to_word={}
for i in range(len(embeddings_keylist)):
  word_to_index[embeddings_keylist[i]]=i
  index_to_word[i]=embeddings_keylist[i]

<class 'dict'>
1999995


In [11]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

# from spacy.lang.en import English
# nlp = English()
# Create a Tokenizer with the default settings for English
# including punctuation rules and exceptions
# tokenizer = nlp.Defaults.create_tokenizer(nlp)



[nltk_data] Downloading package punkt to /home/tjss/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
# # Dummy dataloader
# test_df = pd.read_csv(prefix+"data/new_doc_level_val.csv")
# print(test_df.shape)
# test_df = test_df.iloc[:,1:]
# curr_tex = test_df.iloc[6,0]
# curr_emot = test_df.iloc[6,1]
# curr_embed = np.zeros((300))
# print("curr text", curr_tex)
# print(curr_embed)
# tot_cnt=0
# ign_cnt=0
# for word in word_tokenize(curr_tex):
#   print(word)
#   tot_cnt+=1
#   if word in embeddings.keys():
#     curr_embed = curr_embed+embeddings[word]
#   else:
#     ign_cnt+=1
# print("percentage lost", ign_cnt/tot_cnt*100, "%")
# print(curr_embed.shape)
# curr_embed = curr_embed/tot_cnt
# embed_t = torch.from_numpy(curr_embed)
# print(embed_t)

In [13]:
# device = "cuda" if torch.cuda.is_available() else "cpu"

# kwargs = {'num_workers': 1, 'pin_memory': True} if device=='cuda' else {}

In [14]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

In [15]:
class FastTextVectorLoader(Dataset):
  def __init__(self, csv_file, embeddings):
    self.frame = pd.read_csv(csv_file)
    self.embeddings = embeddings
  def __len__(self):
    return len(self.frame)
  def __getitem__(self, idx):
    if torch.is_tensor(idx):
      idx = idx.tolist()
    curr_tex = self.frame.iloc[idx,1]
    curr_emot = self.frame.iloc[idx,2]
    curr_embed = np.zeros((300))
    tot_cnt=0
    ign_cnt=0
    
    for word in word_tokenize(curr_tex):
      tot_cnt+=1
      if word in embeddings.keys():
        # print(word)
        curr_embed = curr_embed+embeddings[word]
      else:
        ign_cnt+=1
    # print("percentage lost", ign_cnt/tot_cnt*100, "%")
    curr_embed = curr_embed/tot_cnt
    embed_t = torch.from_numpy(curr_embed)
    # embed_t = embed_t.to(torch.device("cuda:0"))
    # curr_emot = curr_emot.cuda()
    data = {'vector':embed_t, 'label':curr_emot}
    return data

In [16]:
train_dataset = FastTextVectorLoader(prefix+"data/new_doc_level_train.csv", embeddings)
val_dataset = FastTextVectorLoader(prefix+"data/new_doc_level_val.csv", embeddings)
test_dataset = FastTextVectorLoader(prefix+"data/new_doc_level_test.csv", embeddings)

In [17]:
train_dataloader = DataLoader(train_dataset, batch_size=64,shuffle=True, num_workers=1)
val_dataloader = DataLoader(val_dataset, batch_size=64,shuffle=True, num_workers=1)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=1)


In [18]:
# testing dataloader
count = 0
for i,batch_data in enumerate(train_dataloader):
  count += 1
  print(batch_data['vector'])
  print(batch_data['vector'].size())
  print(batch_data['label'])
  # batch_data['vector'].cuda()
  # batch_data['vector'].to(device)
  print(batch_data['vector'].is_cuda)
  print(batch_data['label'].is_cuda)
  # print(text)
  # print(text_length)
  if count == 1:
    break


tensor([[-0.0238, -0.0534, -0.0149,  ...,  0.0155,  0.0159,  0.0429],
        [-0.0089, -0.0631,  0.0536,  ..., -0.0025, -0.0675, -0.0217],
        [-0.0044, -0.0028, -0.0125,  ...,  0.1078, -0.0627, -0.0470],
        ...,
        [-0.0104,  0.0169, -0.1725,  ..., -0.0755, -0.0770, -0.0769],
        [ 0.1430,  0.0261, -0.2154,  ..., -0.0013, -0.2866,  0.0638],
        [-0.1244,  0.0112, -0.0266,  ...,  0.0576, -0.0362, -0.0552]],
       dtype=torch.float64)
torch.Size([64, 300])
tensor([0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0,
        0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0])
False
False


In [19]:
 def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    for batch in iterator:
        optimizer.zero_grad()
        text_vector = batch['vector']
        # text_vector = batch['vector'].to(device)
        predictions = model(text_vector)
        
        y_pred_tag = torch.round(torch.sigmoid(predictions))
        
        # labels = batch['label'].to(device)
        labels = batch['label']
        labels = labels.float()
        labels = labels.reshape([ len(labels),1])
        loss = criterion(predictions, labels)
        
        f1 = f1_score(labels.detach().numpy(), y_pred_tag.detach().numpy())
        acc = calc_accuracy(y_pred_tag, labels)


        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator), f1

In [20]:
def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.eval()
    with torch.no_grad():
        for batch in iterator:
            text_vector = batch['vector']
            # text_vector = batch['vector'].to(device)
            predictions = model(text_vector)
            y_pred_tag = torch.round(torch.sigmoid(predictions))
            labels = batch['label']
            # labels = batch['label'].to(device)
            labels = labels.float()
            labels = labels.reshape([len(labels),1])
            loss = criterion(predictions, labels)
            f1 = f1_score(labels, y_pred_tag)
            acc = calc_accuracy(y_pred_tag, labels)
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator), f1

In [21]:
import time
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [22]:
def calc_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    # rounded_preds = torch.round(torch.sigmoid(preds))
    
    correct = (preds == y).float() #convert into float for division 
    # print(correct)
    acc = correct.sum() / len(correct)
    return acc

In [23]:
import torch.nn as nn
import torch.nn.functional as F

class FastText(nn.Module):
    def __init__(self, embedding_length, output_dim):        
        super().__init__()
        
        self.embedding_length = embedding_length
        
        self.output_dim = output_dim

        # self.embedding = nn.Embedding(self.vocab_size, self.embedding_length, padding_idx=pad_idx)
        # self.embedding.weight = nn.Parameter(word_embeddings, requires_grad = False)
        self.fc1 = nn.Linear(self.embedding_length, 512)
        self.fc2 = nn.Linear(512,256)
        self.fc3 = nn.Linear(256,128)
        self.fc4 = nn.Linear(128, output_dim)
        self.relu = nn.ReLU()
        self.dropout1 = nn.Dropout(p=0.1)
        self.dropout2 = nn.Dropout(p=0.4)
        
    def forward(self, text):        
        pooled_drop_op = self.dropout1(text)
        fc1_op = self.fc1(pooled_drop_op)
        fc1_drop_op = self.dropout2(fc1_op)
        fc2_op = self.fc2(fc1_drop_op)
        fc2_drop_op = self.dropout2(fc2_op) 
        fc3_op = self.fc3(fc2_drop_op)       
        return self.fc4(fc3_op)
        # text = self.relu(self.fc1(text))
        # text = self.relu(self.fc2(text))
        # text = self.relu(self.fc3(text))
        # text = self.dropout1(text)
        # return self.fc4(text)
        

In [24]:
EMBEDDING_DIM = 300
OUTPUT_DIM = 1
model = FastText( EMBEDDING_DIM,OUTPUT_DIM)
model = model.double()

In [25]:
import torch.optim as optim
optimizer = optim.Adam(model.parameters(), lr=1e-5)
criterion = nn.BCEWithLogitsLoss()
# criterion = criterion.to(device)

In [26]:
N_EPOCHS =12
best_valid_loss = float('inf')
for epoch in range(N_EPOCHS):
    start_time = time.time()
    train_loss, train_acc, train_f1 = train(model, train_dataloader, optimizer, criterion)
    valid_loss, valid_acc, val_f1 = evaluate(model, val_dataloader, criterion)
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% | Train F1:{train_f1}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% | Val F1:{val_f1}')

Epoch: 01 | Epoch Time: 0m 13s
	Train Loss: 0.679 | Train Acc: 60.71% | Train F1:0.0
	 Val. Loss: 0.669 |  Val. Acc: 61.09% | Val F1:0.0
Epoch: 02 | Epoch Time: 0m 13s
	Train Loss: 0.662 | Train Acc: 61.05% | Train F1:0.0
	 Val. Loss: 0.658 |  Val. Acc: 61.09% | Val F1:0.0
Epoch: 03 | Epoch Time: 0m 14s
	Train Loss: 0.651 | Train Acc: 61.02% | Train F1:0.0
	 Val. Loss: 0.648 |  Val. Acc: 61.02% | Val F1:0.0
Epoch: 04 | Epoch Time: 0m 14s
	Train Loss: 0.638 | Train Acc: 61.30% | Train F1:0.0
	 Val. Loss: 0.633 |  Val. Acc: 62.48% | Val F1:0.08695652173913045
Epoch: 05 | Epoch Time: 0m 14s
	Train Loss: 0.618 | Train Acc: 64.75% | Train F1:0.3076923076923077
	 Val. Loss: 0.612 |  Val. Acc: 66.36% | Val F1:0.3636363636363636
Epoch: 06 | Epoch Time: 0m 14s
	Train Loss: 0.596 | Train Acc: 68.54% | Train F1:0.4615384615384615
	 Val. Loss: 0.592 |  Val. Acc: 69.63% | Val F1:0.2
Epoch: 07 | Epoch Time: 0m 14s
	Train Loss: 0.574 | Train Acc: 71.31% | Train F1:0.3333333333333333
	 Val. Loss: 0.57

In [27]:
# 77% 0.68
test_loss, test_acc, test_f1 = evaluate(model, test_dataloader, criterion)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}% | Test F1: {test_f1}')

Test Loss: 0.530 | Test Acc: 74.13% | Test F1: 0.5333333333333333


In [28]:
torch.save(model.state_dict(),prefix+'data/MeanEmbeddingsFasttext_NRCJoyVsNotJoy_wordLevelBackground(Non Joy Reviews)Shap.pt' )

In [29]:
model1 = FastText(EMBEDDING_DIM,OUTPUT_DIM)
model1.load_state_dict(torch.load(prefix+'data/MeanEmbeddingsFasttext_NRCJoyVsNotJoy_wordLevelBackground(Non Joy Reviews)Shap.pt'))
model1.eval()

FastText(
  (fc1): Linear(in_features=300, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=128, bias=True)
  (fc4): Linear(in_features=128, out_features=1, bias=True)
  (relu): ReLU()
  (dropout1): Dropout(p=0.1, inplace=False)
  (dropout2): Dropout(p=0.4, inplace=False)
)

In [30]:
%pip install shap
import shap

Note: you may need to restart the kernel to use updated packages.


In [31]:
test_df = pd.read_csv(prefix+"data/new_doc_level_train.csv")
test_df.head(5)

,Unnamed: 0,message,emotion_code
0,11390,"io domani volevo vestirmi tutta figa, ma il mi...",0
1,1136,"churchhhh! then exam stuff, and making 30 dang...",1
2,9805,empty fuel tank & a $ load of traffic,1
3,16383,he who cannot forgive others destroys the brid...,1
4,12353,hoje meu dia vai ser study study and study. te...,0


In [32]:
def get_word_embeddings(dataframe_csv):
  vocab={}
  bg = torch.empty((0,300))
  df = pd.read_csv(dataframe_csv)
  for idx, row in df.iterrows():
    curr_text = df.iloc[idx,1]
    curr_label = df.iloc[idx,2]
    if(curr_label==0):
      for word in word_tokenize(curr_text):
        if(word in embeddings.keys() and word not in vocab.keys()):
          vocab[word]=0
          curr_embed = torch.from_numpy(np.array(embeddings[word]))
          curr_embed = curr_embed.reshape((1, len(curr_embed)))
          bg = torch.cat((bg, curr_embed))
#     if(idx==idx_limit):
#       break

  return bg, vocab


In [33]:
new_bg,_ = get_word_embeddings(prefix+"data/new_doc_level_train.csv")
print(new_bg.size())

torch.Size([12733, 300])


In [34]:
e = shap.DeepExplainer(model, new_bg)

In [35]:
explain_examples, vocab1  = get_word_embeddings(prefix+"data/new_doc_level_test.csv")
print(explain_examples.size())

torch.Size([6671, 300])


In [36]:
st = time.time()
shap_vals = e.shap_values(explain_examples.float())
print(time.time()-st)

29267.678956508636


In [37]:
print(len(shap_vals))

6671


In [38]:
print(len(vocab1))

6671


In [39]:

print((shap_vals).shape)

(6671, 300)


In [40]:
shap_means_joy = shap_vals.mean(axis=1)

In [41]:
idx=0
for word in vocab1:
  vocab1[word]=shap_means_joy[idx]
  idx+=1

In [42]:
for word in vocab1:
  print(word, " ", vocab1[word])

heel   -0.007316549357509922
iets   -0.01362928573438027
hulp   -0.01895294701412185
mag   -0.0057381611068327695
wel   -0.004318007380694116
toch   -0.013403446981253703
?   -0.011270338904074787
#   -0.0011276514784195777
apparently   -0.01353606131127325
10   0.015478168786275982
people   -0.002929828664296013
every   0.010919690336288198
year   0.020668873455502806
die   -0.012714510216176039
from   0.005908471102434182
vending   0.013622770824180529
machines   0.005414755937646956
...   0.002644233718005347
it   -0.0018319401669830038
really   -0.004725200768380557
proves   -0.009583367209008193
how   -0.003681668374258561
us   0.0035173392898973713
humans   -0.004246983087925196
can   -0.0018741097212944657
so   0.0021084950042401726
easily   -0.0032759877382987027
.   0.006302927365596638
i   -0.00794202532420643
hate   -0.013628092195083716
when   0.0010199373695634565
go   0.0019339421297829048
to   0.006474893081067023
@   -0.008525341733083981
justinbieber   -0.0119489032470

thing   -0.004628513543223794
fear   -0.015608183957355276
little   0.007915881681925052
inside   0.0019253782330167554
-   0.0031501855994762636
john   -0.0022992643402534253
seriously   -0.008742264221682486
put   -0.00406549960692603
crappy   -0.0015441483819979332
programme   0.019659034871187173
damned   -0.001107313851512698
cauliflower   0.0021251299030305786
them   0.0006125170047356286
spend   0.022363656131350888
hour   0.0311850374317638
your   0.009944401159020249
phone   0.0018626947875387324
end   0.011044921010232446
world   0.007293681259887566
until   0.010138985783293305
next   0.011781557578779257
week   0.025680847626429876
win   -0.0082605588898126
5   0.014902239170900132
special   0.010936482669848846
collector   -0.006616374351714762
items   0.00352177124460159
giving   0.009906527219054903
our   0.013170670513167934
twitter   -0.007650646017004898
oooh   -0.0030977655093036723
wat   -0.019221844843612043
ben   -0.006316468787386818
ik   -0.01975720366113484
wee

fail   -0.015340685728963344
study   0.01668336117464253
friend   -0.0008479736649179415
fl   0.00303282247124741
showed   -0.01119259466537626
door   0.0025574070985565443
step   0.0054931595496330735
tried   -0.01122401738227139
tickets   0.001473052921238128
jerry   -0.007907518682495986
seinfeld   -0.011174231199369755
confirmed   -0.005736975900655856
looks   -0.0035130546196494067
nba   -0.013717666384499803
schedule   0.028778998068242297
daily   0.022508939349817826
had   -0.006254072793840259
hair   0.006198809288384963
kelli   0.004259400451897892
longer   0.0069474317100423795
exists   -0.008614418370895044
called   -0.006480615239328456
fleet   0.0026330702823365963
irish   0.0025732753480225867
pub   0.006179795273528488
food   0.013159591542584632
leuk   -0.0064094528911606465
gisteravond   -0.010125114537010472
laat   -0.007221463106815746
lead   -0.0005256365470588791
mijn   -0.003273124668374038
mailbox   0.006415951962954658
briljant   -0.006395581242968717
idee   -0.

nd   0.0027061494421579467
moody   0.00540177049533261
reason   -0.007165954415991084
help   0.007135911779434344
bought   -0.007405904533228266
some   0.0046735358612443375
crackers   0.013037129725724485
usually   0.0007983808160418811
nigger   -0.0153123033603261
6   0.016568162524923866
years   0.010043140898324565
ex   -0.011163624883095141
died   -0.008053464248885643
tested   0.002594306184974592
came   -0.0040020748732593585
cherche   -0.009360720460292179
ce   0.0010765121985454526
samedi   0.011690694868726433
des   -0.006337495782748392
merci   -0.0019592371823870885
shoes   0.002688404381595533
em   -0.0042672565436381125
element   -0.0014936910948720199
pause   0.013176537466220885
perd   -0.006869350940475752
district   0.007821230337138359
coupe   0.0006711522568492704
france   -0.0052275318385271875
1-0   -0.006200828565674529
birth   0.014554286098990244
control   -0.002270893258538029
starving   -0.0008894727708511001
kidding   -0.007082007943842725
echt   -0.01240068

mouse   -0.00043822234308707686
staying   0.009274350885463896
cat   -0.0011024155949194833
whereas   -0.003176538691059614
humming   0.02741446963445822
probably   -0.010386258707472139
ican   -0.0003896269789983971
smdh   -0.023041657694863167
fucked   -0.008199706329652837
read   0.0033091568828450078
iknow   -0.013033683729174562
vela   0.0036542715848725186
scored   0.0010542872153755105
chip   -0.002133424522518908
uh   -0.013687670565607919
visit   0.01243893110277114
doin   0.00163657711832386
dew   0.013506409559629269
chose   -0.006185720215663104
remind   0.0010339067366175198
scripture   0.01566792772066063
serving   0.019963064801605424
apply   0.003410199772306576
confusion   -0.0074832800714674085
childhood   0.007726581953603547
ending   0.0040980267108568945
doesnt   -0.018589008654166652
kinda   -0.012188719096723139
mad   -0.007192935927639909
deserve   -0.001299579727821335
happiness   0.025679439060988744
success   0.00758736972143956
gotten   -0.007374922916720295

...

sell   -0.006889971211736516
yhu   -0.017270093836730775
bone   0.002332152205183231
chex   0.004410271137572736
mix   0.010677715956303388
opened   0.0019875666130974713
5pm   0.0257046883957653
hahahaha   -0.011096934630506145
messages   0.007257299758291989
constant   0.008616849761232314
so-called   -0.0035490584593683067
progressive   0.005139622130608121
wh   -0.0025322991137282803
lack   -0.012798256999490656
ability   0.003712736154938487
spite   -0.0059449938116749495
costa   0.007468119240747419
ton   0.0037130092779737874
waffle   0.00930308151931375
spanish   0.0072402333392676595
respond   -0.005916208532890829
gramatically   -0.013808720410324596
correct   -0.009611097722564953
club   0.0035003351327959832
penguin   0.005814106154974268
freezes   -0.008954294501175917
fans   -0.014460961836738307
wae   -0.010692062603485387
boiz   -0.014916143449543041
clobber   -0.004551058360047411
runs   0.0070521767748871625
rb   -0.007698519274969516
virgin   0.0027668051197606027
be

sudden   -0.0076884362846465805
omelette   0.008861031891453269
roll   0.009412216093119749
unpatriotic   -0.020720206593803005
civil   0.008614135617402591
liberties   -0.0068752900266573404
exchange   0.010691900069032943
debate   0.0014031276794065398
allows   0.004133763703023458
lonely   0.004271060188507806
enraged   -0.022867210357403002
belittled   -0.018083607245828724
vous   -0.007283960097791457
devez   -0.008834042809195823
utiliser   -0.01086783241579766
internet   0.004158175860883782
explorer   0.0008937162386811604
visualiser   0.006508806351728388
billy   -0.004062831721161484
talent   0.003394191877610562
definitely   -0.0031647752378748226
effort   0.007567600402427187
twentyonepilots   -0.00571609344866557
jealousy   -0.009087554047883452
leslie   0.0004623325918858967
officemate   0.001744530229501969
holds   -0.0019820248040941725
stops   0.0024310626261088613
potential   -0.003195321551921598
forces   -0.0031794020584572187
fooled   -0.01941938329597995
raging   

bcs   -0.010493224283878328
rug   0.0037488905162457323
vast   0.002251822985057989
kun   -0.01005183561241919
haar   -0.011643919833632747
misschien   -0.022914071467436153
brengen   -0.011216930299289692
mtv   -0.013303642284169243
barfing   0.015569636395557064
popped   -0.0004121372332063331
recieving   0.00519939120372488
4s   0.005307029913208586
verizon   -0.005374036074144262
congratulating   0.012224731064666301
dusted   0.01909877819225761
concentrate   0.021841636393818056
design   0.008859823760809432
younger   -0.0017193037013748754
pierced   -0.0034197026233189804
dress   0.00527613348783684
asos   -0.0032263894195610214
petite   0.000622363946857856
range   0.014867877331476077
skink   -0.0029698319578548337
lick   0.00351951469507292
tongue   0.0033897779735627496
mimicking   -0.005125673319745085
joseph   -0.0006277713505115467
chilton   -0.002874206223164172
pearce   0.0006433683222902103
pair   0.005467057455301051
jeans   0.0033289242765995697
embarrassing   -0.0177

jari   0.003684141486401702
ku   -0.00034557807871085536
kk   -0.008983422783918954
decorate   0.028981265462969905
blackberry   0.004694663564202495
moses   -0.004151361558671168
ironing   0.025999564726635508
shirts   -0.0005642638112381685
messed   -0.009452776728508545
suitcase   0.006687233938698457
cartoon   -0.0024270279925322396
network   0.009397545480582305
nickelodeon   -0.0074919864416665965
........   0.0011794295259371184
21   0.009506347165082672
managed   0.0018293496701214432
5000   0.0076028230497623755
wack   -0.008672358438450007
nerdy   -0.000940550062681256
squeeze   0.016730845444748663
baggy   -0.00506467334571883
army   -0.0031546124137852463
wrapping   0.028457042142764722
hippier   -0.004060008388877063
lyst   0.005320430805752611
rive   0.004916022556497815
hodet   -0.025204572866649257
av   -0.0039162947783256815
periods   0.0230419655290531
depth   0.004118792607903572
truly   0.004341331245457857
grades   0.0216346119638542
itry   -0.0006807485974598016
r

mattress   0.006733386064757161
boo   -0.006608348931999614
points   0.0059729821053245875
literature   0.012167671163038933
rio   0.007212338164335897
q-tip   0.009640170475865956
sem   0.01033960002736671
grana   0.004027263279865061
bh   -0.005808264183032351
bosom   0.00827425319428957
pissed   -0.01870197954378204
low   0.005241850751706495
disney   -0.0033881074727410325
unltd   0.01837157028959394
caused   -0.017741695590480902
user   -0.006397589077238243
25.-   0.005678927618392587
ski   0.020553481006292312
premiers   0.0021001987852938453
8h30   0.029121167197686842
caisses   0.0075945061753816745
crans   0.009149657572650677
refreshing   0.015513131156766048
flat   0.00513536706837549
trumpet   0.011677176840011466
five   0.017377304511907384
naaah   -0.019080933885992794
nothig   -0.0053606864756092
obstacle   -0.004029877078056458
stands   -0.0018314757438287314
progress   0.012558976545863828
orgasm   0.007868844956095272
toevallig   -0.026992924193134038
paar   -0.01295

In [43]:
df = pd.DataFrame(list(vocab1.items()),columns = ['column1','column2']) 

In [44]:
lexicon_df_joy = df.sort_values(by='column2')

In [45]:
lexicon_df_joy.head(50)

,column1,column2
5306,friendzoned,-0.037401
3938,jobro,-0.029772
3960,illigal,-0.029086
4221,konami,-0.028685
4122,artest,-0.028617
1109,iemand,-0.028536
1389,reageren,-0.028274
4049,tegen,-0.028266
3214,kijkt,-0.027478
5890,beiber,-0.027241


In [46]:
lexicon_df_joy.tail(50)

,column1,column2
297,day,0.028016
1845,warm,0.028063
6074,healing,0.028076
2187,busy,0.028236
5372,wrapping,0.028457
3869,ngayong,0.028748
6554,calendar,0.028774
897,schedule,0.028779
5353,decorate,0.028981
6116,8h30,0.029121


In [47]:
lexicon_df_joy.to_csv(prefix+"results/MeanEmbeddingsFasttext_NRCJoyVsNotJoy_wordLevelBackground(Non Joy Reviews_ALL)Shap.csv")
